# Emprical Asset Pricing - Problem Set 4: Variance Risk Premium
Group Member: Victor Xiao, Zi Wang, Sonny Song
Discussed with HaoYang Sun, Wenxin Zeng, Xiaoyu Zhang

### 1. The Term Structure of Variance Risk Premium

#### 1.a OptionMetrics (in WRDS) Data Preprocesssing

In [42]:
import numpy as np
import pandas as pd
import os
import statsmodels.api as sm
from scipy import optimize
import datetime as dt
from datetime import datetime
import wrds
import matplotlib.pyplot as plt
import tqdm 
from statsmodels.iolib.summary2 import summary_col
import pandas_datareader
from scipy import stats

db = wrds.Connection(wrds_username = 'vince_solis')

Loading library list...
Done


In [11]:
def clean_option_data(df):
    """Clean and preprocess option data."""
    df = df[['date', 'cp_flag', 'strike_price', 'exdate', 'best_bid', 'best_offer', 'optionid']]
    df['date'], df['exdate'] = pd.to_datetime(df['date']), pd.to_datetime(df['exdate'])
    df['strike_price'] /= 1000
    df['Time_to_maturity'] = (df['exdate'] - df['date']).dt.days
    return df

def fetch_options_data(start_year=1996, end_year=datetime.now().year):
    """Fetch and concatenate options data for a range of years."""
    options_df = pd.DataFrame()
    for year in tqdm.tqdm(range(start_year, end_year)):
        query = f"SELECT * FROM optionm.opprcd{year} WHERE secid = 108105"
        df = db.raw_sql(query)
        df = clean_option_data(df)
        options_df = pd.concat([options_df, df])
    return options_df

In [12]:
# Retrieve options data
options_df = fetch_options_data()

# Fetch S&P500 spot index data
spx_spot_query = "SELECT date, spindx FROM crsp_a_stock.dsi WHERE date BETWEEN '1996-01-01' AND '2022-10-31'"
spx_spot = db.raw_sql(spx_spot_query)
spx_spot['date'] = pd.to_datetime(spx_spot['date'])
options_df = pd.merge(options_df, spx_spot, how='left', on='date')

# Fetch and process risk-free rates
rf_query = """
    SELECT DISTINCT dateff AS date, rf FROM ff_all.factors_monthly
    WHERE dateff >= '1980-01-01' ORDER BY date
"""
rf = db.raw_sql(rf_query)
rf['date'] = pd.to_datetime(rf['date'])
rf['year'], rf['month'] = rf['date'].dt.year, rf['date'].dt.month

# Calculate Moneyness ratio
options_df['Moneyness_ratio'] = options_df['strike_price'] / options_df['spindx']

# Generate trading calendar
trading_calendar = sorted(options_df['date'].unique())

  0%|          | 0/28 [00:00<?, ?it/s]C:\Users\ConquerV\AppData\Local\Temp\ipykernel_41060\1052777930.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['date'], df['exdate'] = pd.to_datetime(df['date']), pd.to_datetime(df['exdate'])
C:\Users\ConquerV\AppData\Local\Temp\ipykernel_41060\1052777930.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['strike_price'] /= 1000
C:\Users\ConquerV\AppData\Local\Temp\ipykernel_41060\1052777930.py:6: SettingWithCopyWarning: 
A value is trying to be set on a cop

In [ ]:
def preproc_options_data(options_df, freq):
    # Define cutoff days for different frequencies
    cutoff_days = {'1M': (30, 60), '1Y': (365, 545)}
    if freq not in cutoff_days:
        raise ValueError(f"Frequency {freq} is not recognized.")

    # Announce the data cleaning process
    print(f"Cleaning data for options expiring in {freq}\n")
    
    # Define minimum and maximum days to maturity based on the frequency
    min_days, max_days = cutoff_days[freq]
    # Filter the dataframe for the specified frequency
    options_within_range = options_df[(options_df['Time_to_maturity'] >= min_days)]
    options_within_range['Moneyness_deviation'] = (options_within_range['Moneyness_ratio'] - 1).abs()

    # Apply moneyness deviation cutoff
    options_within_range = options_within_range[options_within_range['Moneyness_deviation'] <= 0.05]

    # Group by date and call/put flag, and find the option with the minimum time to maturity
    grouped = options_within_range.groupby(['date', 'cp_flag'])
    min_time_to_maturity = grouped['Time_to_maturity'].transform('min')
    options_min_maturity = options_within_range[options_within_range['Time_to_maturity'] == min_time_to_maturity]

    # Find the option with the minimum moneyness deviation
    min_moneyness_deviation = grouped['Moneyness_deviation'].transform('min')
    options_closest_to_atm = options_min_maturity[options_min_maturity['Moneyness_deviation'] == min_moneyness_deviation]

    # Select options with the lowest strike price when there are multiple options for a date
    options_lowest_strike = options_closest_to_atm.sort_values(by='strike_price').groupby(['date', 'cp_flag']).head(1)

    # Select options with the highest best offer, likely to be non-standard options
    options_highest_offer = options_lowest_strike.sort_values(by='best_offer', ascending=False).groupby(['date', 'cp_flag']).head(1)

    # Deduplicate the dataframe
    options_deduped = options_highest_offer.drop_duplicates(subset=['date', 'cp_flag', 'strike_price', 'exdate'])

    # Check for the existence of both a call and put option
    options_deduped['n_Opt'] = options_deduped.groupby(['date'])['optionid'].transform('nunique')
    options_final = options_deduped[options_deduped['n_Opt'] == 2]

    # Assert that we have sufficient date coverage
    assert options_final['date'].nunique() / len(trading_calendar) >= 0.95, "Insufficient date coverage"

    # Group by year and month to create a monthly dataset
    options_final['year'] = options_final['date'].dt.year
    options_final['month'] = options_final['date'].dt.month
    monthly_options = options_final.groupby(['year', 'month', 'cp_flag']).last().reset_index()

    # Display summaries and the head of the dataset
    print(monthly_options['Time_to_maturity'].describe())
    print(monthly_options['Moneyness_deviation'].describe())
    
    final_dataset = monthly_options.reset_index(drop=True).drop(columns=[
        'Moneyness_deviation', 'Time_to_maturity', 'Min_Time_to_maturity', 
        'Min_Moneyness_deviation', 'min_strike', 'min_best_offer', 'n_Opt'
    ])

    display(final_dataset.head())
    return final_dataset


In [66]:
def preproc_options_data(options_df, freq):
    # Define cutoff days for different frequencies
    cutoff_days = {'1M': (30, 60), '1Y': (365, 545)}
    if freq not in cutoff_days:
        raise ValueError(f"Frequency {freq} is not recognized.")

    # Announce the data cleaning process
    print(f"Cleaning data for options expiring in {freq}\n")
    
    # Define minimum and maximum days to maturity based on the frequency
    min_days, max_days = cutoff_days[freq]
    # Filter the dataframe for the specified frequency
    options_df_freq = options_df[(options_df['Time_to_maturity'] >= min_days)]
    options_df_freq['Moneyness_deviation'] = (options_df_freq['Moneyness_ratio'] - 1).abs()

    # Apply moneyness deviation cutoff
    options_df_freq = options_df_freq[options_df_freq['Moneyness_deviation'] <= 0.05]

    options_df_freq['Min_Time_to_maturity'] = options_df_freq.groupby(['date', 'cp_flag'])['Time_to_maturity'].transform('min')
    options_df_freq_consid = options_df_freq[options_df_freq['Time_to_maturity'] == options_df_freq['Min_Time_to_maturity']].copy()

    options_df_freq_consid['Min_Moneyness_deviation'] = options_df_freq_consid.groupby(['date', 'cp_flag'])['Moneyness_deviation'].transform('min')
    options_df_freq_consid = options_df_freq_consid[options_df_freq_consid['Moneyness_deviation'] == options_df_freq_consid['Min_Moneyness_deviation']].copy()

    # If more than 1 call and put pair, select the lower strike (that's what CBOE does!)
    options_df_freq_consid['min_strike'] = options_df_freq_consid.groupby(['date'])['strike_price'].transform('min')
    options_df_freq_consid = options_df_freq_consid[options_df_freq_consid['strike_price'] == options_df_freq_consid['min_strike']]

        # Select options with the highest best offer, likely to be non-standard options
    options_df_freq_consid['min_best_offer'] = options_df_freq_consid.groupby(['date', 'cp_flag'])['best_offer'].transform('max')
    options_df_freq_consid = options_df_freq_consid[options_df_freq_consid['min_best_offer'] == options_df_freq_consid['best_offer']]

    options_df_freq_consid = options_df_freq_consid.drop_duplicates(subset = ['date', 'cp_flag','strike_price', 'exdate' ])

    # Deduplicate the dataframe
    options_deduped = options_df_freq_consid.drop_duplicates(subset=['date', 'cp_flag', 'strike_price', 'exdate'])

    options_deduped['n_Opt'] = options_deduped.groupby(['date'])['optionid'].transform('nunique')

    # some strikes may have only a call or a put but not both (AFAIK this impacts only 1 option in 1996, low liquidity)
    options_final = options_deduped[options_deduped['n_Opt'] == 2]

    # Assert that we have sufficient date coverage
    assert options_final['date'].nunique() / len(trading_calendar) >= 0.95, "Insufficient date coverage"

    # Group by year and month to create a monthly dataset
    options_final['year'] = options_final['date'].dt.year
    options_final['month'] = options_final['date'].dt.month
    monthly_options = options_final.groupby(['year', 'month', 'cp_flag']).last().reset_index()

    # Display summaries and the head of the dataset
    print(monthly_options['Time_to_maturity'].describe())
    print(monthly_options['Moneyness_deviation'].describe())
    
    final_dataset = monthly_options.reset_index(drop=True).drop(columns=[
        'Moneyness_deviation', 'Time_to_maturity', 'Min_Time_to_maturity', 
        'Min_Moneyness_deviation', 'min_strike', 'min_best_offer', 'n_Opt'
    ])

    display(final_dataset.head())
    print("\n\n\n")

    return final_dataset

In [70]:
options_clean_1M = preproc_options_data(options_df, '1M')

Cleaning data for options expiring in 1M



C:\Users\ConquerV\AppData\Local\Temp\ipykernel_41060\878926887.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  options_df_freq['Moneyness_deviation'] = (options_df_freq['Moneyness_ratio'] - 1).abs()


count    644.000000
mean      41.835404
std        9.499820
min       30.000000
25%       31.000000
50%       47.000000
75%       50.000000
max       80.000000
Name: Time_to_maturity, dtype: float64
count    644.000000
mean       0.001831
std        0.002413
min        0.000000
25%        0.000465
50%        0.000964
75%        0.001878
max        0.013726
Name: Moneyness_deviation, dtype: float64


C:\Users\ConquerV\AppData\Local\Temp\ipykernel_41060\878926887.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  options_final['year'] = options_final['date'].dt.year
C:\Users\ConquerV\AppData\Local\Temp\ipykernel_41060\878926887.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  options_final['month'] = options_final['date'].dt.month


,year,month,cp_flag,date,strike_price,exdate,best_bid,best_offer,optionid,spindx,Moneyness_ratio
0,1996,1,C,1996-01-31,635.0,1996-03-16,10.250,11.000,11081115.0,636.02,0.998396
1,1996,1,P,1996-01-31,635.0,1996-03-16,8.125,8.250,11076729.0,636.02,0.998396
2,1996,2,C,1996-02-29,640.0,1996-04-20,13.750,14.500,11377327.0,640.43,0.999329
3,1996,2,P,1996-02-29,640.0,1996-04-20,13.625,14.000,10101560.0,640.43,0.999329
4,1996,3,C,1996-03-29,645.0,1996-05-18,17.375,18.125,10629651.0,645.50,0.999225


#### 1.b Straddle Return with maturity closest but longer than one Month.

In [71]:
def nearest(items, pivot):
    return min([i for i in items if i <= pivot], key = lambda x: abs(x - pivot))

def get_trading_returns(options_clean_nM):
    options_clean_nM_iod = options_clean_nM['optionid'].tolist()
    options_clean_nM['sell_date'] = options_clean_nM['date'] + dt.timedelta(30) + pd.offsets.BDay(0)

    holiday_sell = sorted(set(options_clean_nM['sell_date'].drop_duplicates()) - set(trading_calendar))
    holiday_adjusted_sell_date = [nearest(trading_calendar, holiday_sale) for holiday_sale in holiday_sell]
    holiday_sell_map = dict(zip(holiday_sell, holiday_adjusted_sell_date))

    # adjust dates
    options_clean_nM['sell_date_adj'] = options_clean_nM['sell_date'].map(holiday_sell_map)
    options_clean_nM['sell_date_adj'] = options_clean_nM['sell_date_adj'].fillna(options_clean_nM['sell_date'])

    assert set(options_clean_nM['sell_date_adj'].drop_duplicates()) - set(trading_calendar) == set()

    options_df_sell = options_df[(options_df['optionid'].isin(options_clean_nM_iod)) & (options_df['date'].isin(options_clean_nM['sell_date_adj'].tolist()))]

    assert len(options_df_sell['optionid'].drop_duplicates()) == len(options_clean_nM['optionid'].drop_duplicates())
    assert len(options_df_sell['date'].drop_duplicates()) == len(options_clean_nM['sell_date_adj'].drop_duplicates())

    assert options_df_sell.shape == options_df_sell.drop_duplicates(subset = ['date', 'optionid']).shape

    options_clean_nM = pd.merge(options_clean_nM, options_df_sell[['date', 'optionid', 'best_bid', 'best_offer', 'spindx']], left_on = ['sell_date_adj', 'optionid'], right_on = ['date', 'optionid'], how = 'left', suffixes = ('_long', '_short'))

    options_clean_nM = options_clean_nM[['date_long', 'date_short', 'cp_flag', 'optionid', 'best_bid_long', 'best_offer_long', 'best_bid_short', 'best_offer_short']].copy()

    # remove december 2021 contracts (since you can't hold them for a month)
    options_clean_nM = options_clean_nM[options_clean_nM['date_long'].dt.date <= dt.date(2021, 11, 30)]

    options_clean_nM_call = options_clean_nM[options_clean_nM['cp_flag'] == 'C'].copy()
    options_clean_nM_put = options_clean_nM[options_clean_nM['cp_flag'] == 'P'].copy()

    assert options_clean_nM_call.shape == options_clean_nM_put.shape

    options_clean_nM_final = pd.merge(options_clean_nM_call.drop(columns = ['cp_flag']), options_clean_nM_put[['date_long', 'date_short', 'optionid', 'best_bid_long', 'best_offer_long', 'best_bid_short', 'best_offer_short']], on = ['date_long', 'date_short'], how = 'left', suffixes = ('_call', '_put'))

    options_clean_nM_final['long_S_price'] = options_clean_nM_final['best_offer_long_call'] + options_clean_nM_final['best_offer_long_put']
    options_clean_nM_final['short_S_price'] = options_clean_nM_final['best_bid_short_call'] + options_clean_nM_final['best_bid_short_put']

    # Not tradable, but this is what they use in the paper.
    options_clean_nM_final['long_S_price_mid'] = 0.5 * (options_clean_nM_final['best_offer_long_call'] + options_clean_nM_final['best_offer_long_put'] + options_clean_nM_final['best_bid_long_call'] + options_clean_nM_final['best_bid_long_put'])
    options_clean_nM_final['short_S_price_mid'] = 0.5 * (options_clean_nM_final['best_bid_short_call'] + options_clean_nM_final['best_bid_short_put'] + options_clean_nM_final['best_offer_short_call'] + options_clean_nM_final['best_offer_short_put'])

    options_clean_nM_final['S_return_correct'] = options_clean_nM_final['short_S_price'] / options_clean_nM_final['long_S_price'] - 1
    options_clean_nM_final['S_return'] = options_clean_nM_final['short_S_price_mid'] / options_clean_nM_final['long_S_price_mid'] - 1

    options_clean_nM_final['year'] = options_clean_nM_final['date_long'].dt.year
    options_clean_nM_final['month'] = options_clean_nM_final['date_long'].dt.month

    options_clean_nM_final['year_short'] = options_clean_nM_final['date_short'].dt.year
    options_clean_nM_final['month_short'] = options_clean_nM_final['date_short'].dt.month

    options_clean_nM_final = pd.merge(options_clean_nM_final, rf, on = ['year', 'month'], how = 'left')

    options_clean_nM_final['S_return_excess'] = options_clean_nM_final['S_return'] - options_clean_nM_final['rf']
    options_clean_nM_final['S_return_correct_excess'] = options_clean_nM_final['S_return_correct'] - options_clean_nM_final['rf']

    return options_clean_nM_final

In [72]:
options_clean_1M_final = get_trading_returns(options_clean_1M)

In [73]:
mean_1M, std_1M = options_clean_1M_final['S_return_excess'].mean() * 12, options_clean_1M_final['S_return_excess'].std() * np.sqrt(12)
sharpe_1M = mean_1M / std_1M

In [74]:
straddle_returns_1M = pd.DataFrame([[mean_1M, std_1M, sharpe_1M]], columns = ['Average Return', 'Volatility', 'Sharpe Ratio']).T.round(3)
straddle_returns_1M

,0
Average Return,-1.269
Volatility,1.762
Sharpe Ratio,-0.720


#### 1.c Straddle Return with maturity closest but longer than one year. 

#### 1.d Risk Premium, volatility, Sharpe ratio of the straddle.

In [75]:
options_clean_1Y = preproc_options_data(options_df, '1Y')

Cleaning data for options expiring in 1Y

count    644.000000
mean     444.791925
std       59.755777
min      365.000000
25%      386.000000
50%      443.000000
75%      502.000000
max      687.000000
Name: Time_to_maturity, dtype: float64
count    644.000000
mean       0.004877
std        0.004345
min        0.000089
25%        0.001673
50%        0.003833
75%        0.006698
max        0.029205
Name: Moneyness_deviation, dtype: float64


C:\Users\ConquerV\AppData\Local\Temp\ipykernel_41060\878926887.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  options_df_freq['Moneyness_deviation'] = (options_df_freq['Moneyness_ratio'] - 1).abs()
C:\Users\ConquerV\AppData\Local\Temp\ipykernel_41060\878926887.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  options_final['year'] = options_final['date'].dt.year
C:\Users\ConquerV\AppData\Local\Temp\ipykernel_41060\878926887.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a 

,year,month,cp_flag,date,strike_price,exdate,best_bid,best_offer,optionid,spindx,Moneyness_ratio
0,1996,1,C,1996-01-31,625.0,1997-06-21,56.000,57.000,10279069.0,636.02,0.982674
1,1996,1,P,1996-01-31,625.0,1997-06-21,22.625,23.625,11659072.0,636.02,0.982674
2,1996,2,C,1996-02-29,650.0,1997-06-21,46.625,48.625,10608799.0,640.43,1.014943
3,1996,2,P,1996-02-29,650.0,1997-06-21,32.750,33.500,10959659.0,640.43,1.014943
4,1996,3,C,1996-03-29,650.0,1997-06-21,54.750,56.750,10608799.0,645.50,1.006971


In [76]:
options_clean_1Y_final = get_trading_returns(options_clean_1Y)

In [77]:
mean_1Y, std_1Y = options_clean_1Y_final['S_return_excess'].mean() * 12, options_clean_1Y_final['S_return_excess'].std() * np.sqrt(12)
sharpe_1Y = mean_1Y / std_1Y
straddle_returns_1Y = pd.DataFrame([[mean_1Y, std_1Y, sharpe_1Y]], columns = ['Average Return', 'Volatility', 'Sharpe Ratio']).T.round(3)
straddle_returns_1Y

,0
Average Return,-0.014
Volatility,0.258
Sharpe Ratio,-0.053


#### 1.e GMM Test: Sharpe ratio increase with Maturity

In the context of hypothesis testing, our null hypothesis $H_0$ postulates that the expected excess return per unit of risk for the short-maturity portfolio is less than or equal to that of the long-maturity portfolio:

$$
H_0: \frac{E(R^e_{1t})}{\sigma(R^e_{1t})} \leq \frac{E(R^e_{2t})}{\sigma(R^e_{2t})}
$$

Conversely, the alternative hypothesis $H_1$ contends that the short-maturity portfolio yields a greater expected excess return per unit of risk compared to the long-maturity portfolio:

$$
H_1: \frac{E(R^e_{1t})}{\sigma(R^e_{1t})} > \frac{E(R^e_{2t})}{\sigma(R^e_{2t})}
$$

To formalize the testing procedure, we introduce the moments $\mu_{ij} = E\left((R^e_{it})^j\right)$. Our aim is to ascertain the asymptotic distribution of the estimator $\hat{\tau}$, which is based on these moments. The estimator $\tau$ itself is defined as:

$$
\tau = \frac{\mu_{11} - \mu_{21}}{\sqrt{\mu_{12} - \mu^2_{11}} - \sqrt{\mu_{22} - \mu^2_{21}}}.
$$

The vector $\mu = (\mu_{11}, \mu_{12}, \mu_{21}, \mu_{22})$ is estimated via the Generalized Method of Moments (GMM). The estimator of $\mu$, denoted as $\hat{\mu}$, has an asymptotic normal distribution:

$$
\sqrt{T}(\hat{\mu} - \mu) \overset{d}{\to} N(0, V),
$$

where $V$ represents the asymptotic variance of the estimator and is given by the infinite sum of the autocovariance matrices of the moment conditions:

$$
V = \sum_{j=-\infty}^{\infty} E(f_tf_{t-j}').
$$

Employing the delta method, we deduce that the estimator $\hat{\tau}$ converges in distribution to a normal distribution centered at zero with variance determined by the gradient matrix $G$:

$$
\sqrt{T}(\hat{\tau} - \tau) \overset{d}{\to} N(0, G'VG),
$$

where $G$ is the Jacobian matrix of partial derivatives of $\tau$ with respect to $\mu$, computed as:

$$
G = \frac{\partial \tau}{\partial \mu} = \begin{bmatrix}
(\mu_{12} - \mu_{11}^2)^{-1/2} + \mu_{11}(\mu_{12} - \mu_{11}^2)^{-3/2} \\
 -0.5\mu_{11}(\mu_{12} - \mu_{11}^2)^{-3/2} \\
-(\mu_{22} - \mu_{21}^2)^{-1/2} - \mu_{21}(\mu_{22} - \mu_{21}^2)^{-3/2} \\ 
0.5\mu_{21}(\mu_{22} - \mu_{21}^2)^{-3/2}
\end{bmatrix},
$$

The matrix $G$ is approximated by $\hat{G}$, which is calculated by replacing the population moments with their sample counterparts.

For the variance estimator $V$, we utilize a $k$-month Newey-West estimator to account for potential autocorrelation and heteroskedasticity in the error terms:

$$
\hat{V} = \sum_{j=-k}^{k} \left( \frac{k + 1 - |j|}{k + 1} \right) \frac{1}{T} \sum_{t=1+j}^{T-\min\{0,j\}} f_tf_{t-j}'.
$$

Finally, our test statistic $\hat{\tau}$ is calculated using the formula:

$$
\hat{\tau} = (\hat{G}'\hat{V}^{-1}\hat{G}/T)^{-1/2}
$$

This test statistic follows a standard normal distribution, enabling us to conduct hypothesis tests about the relative expected excess returns per unit of risk for different maturity portfolios.


In [96]:
def gamma(phi, j):
    
    T = len(phi)
    
    if j == 0:
        output = (1 / T) * (phi.T @ phi)
    elif j > 0:
        phi_lag = phi.shift(j)
        ix = phi_lag.dropna().index
        output = (1 / T) * (phi.loc[ix].T @ phi_lag.loc[ix]) 
    else:
        raise ValueError
        
    return output
def neweyWest(phi, k):
    
    assert k > 0
    gamma_s = gamma(phi, 0)
    
    for i in range(1, k + 1):
        wt = 1 - (i / (k + 1))
        gamma_s += wt * (gamma(phi, i) + gamma(phi, i).T)
    return gamma_s

In [97]:
# Preprocessing to get excess returns sign adjusted
options_clean_1M_final['S_return_excess_sgn'] = -options_clean_1M_final['S_return_excess']
options_clean_1Y_final['S_return_excess_sgn'] = -options_clean_1Y_final['S_return_excess']

# Create moments dataframe and drop missing values
moments = pd.DataFrame({
    '1M_sgn': options_clean_1M_final['S_return_excess_sgn'],
    '1M_sgn_sq': options_clean_1M_final['S_return_excess_sgn'] ** 2,
    '1Y_sgn': options_clean_1Y_final['S_return_excess_sgn'],
    '1Y_sgn_sq': options_clean_1Y_final['S_return_excess_sgn'] ** 2
}).dropna()

# Calculate the mean of the moments
moments_mean = moments.mean()
moment_errors = moments - moments_mean
moments_mean_list = moments_mean.values.tolist()

# Apply Newey-West estimator
V_matrix = neweyWest(moment_errors, 2)

# Calculate gradient vector
G_vector = [
    (moments_mean_list[1] - moments_mean_list[0] ** 2) ** (-0.5) + moments_mean_list[0] ** 2 * (moments_mean_list[1] - moments_mean_list[0] ** 2) ** (-1.5),
    -0.5 * moments_mean_list[0] * (moments_mean_list[1] - moments_mean_list[0] ** 2) ** (-1.5),
    -(moments_mean_list[3] - moments_mean_list[2] ** 2) ** (-0.5) + moments_mean_list[2] ** 2 * (moments_mean_list[3] - moments_mean_list[2] ** 2) ** (-1.5),
    -0.5 * moments_mean_list[2] * (moments_mean_list[3] - moments_mean_list[2] ** 2) ** (-1.5)
]

# Calculate Tau statistic
Tau_stat = (moments_mean_list[0] / np.sqrt(moments_mean_list[1] - moments_mean_list[0] ** 2) -
            moments_mean_list[2] / np.sqrt(moments_mean_list[3] - moments_mean_list[2] ** 2))

V = np.array(V_matrix)
G = np.array(G_vector)

# Calculate test statistic
test_statistic = Tau_stat / np.sqrt(G.T @ V @ G / len(moments))


print(f"Test statistic is: {test_statistic:.2f}; we Reject the Null Hypothesis" if test_statistic > 1.96 else f"Test statistic is: {test_statistic:.2f}; we Fail to Reject the Null Hypothesis")

Test statistic is: 2.74; we Reject the Null Hypothesis


#### 1.f Mean returns and Sharpe. Median VIX as demacation line, what do you find?

In [80]:
vrp = pd.read_table('data/VRPtable.txt', sep = '\s+')
vrp.columns = vrp.columns.str.lower()

VIX = vrp[['year', 'month', 'iv']].copy()
VIX['VIX'] = VIX['iv'] / np.sqrt(12)

VIX['regime'] = np.nan
VIX.loc[VIX['VIX'] <= VIX['VIX'].median(), 'regime'] = 'Low Vol'
VIX.loc[VIX['VIX'] > VIX['VIX'].median(), 'regime'] = 'High Vol'

C:\Users\ConquerV\AppData\Local\Temp\ipykernel_41060\765478009.py:8: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Low Vol' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  VIX.loc[VIX['VIX'] <= VIX['VIX'].median(), 'regime'] = 'Low Vol'


In [81]:
options_clean_1M_final = pd.merge(options_clean_1M_final, VIX.drop(columns = ['iv']), left_on = ['year_short', 'month_short'], right_on = ['year', 'month'], how = 'left')
options_clean_1Y_final = pd.merge(options_clean_1Y_final, VIX.drop(columns = ['iv']), left_on = ['year_short', 'month_short'], right_on = ['year', 'month'], how = 'left')

options_clean_1M_final = options_clean_1M_final.drop(columns = ['year_y', 'month_y'])
options_clean_1Y_final = options_clean_1Y_final.drop(columns = ['year_y', 'month_y'])

In [82]:
def summarize_regime(df):
    excess_ret = df.groupby('regime')[['S_return_excess']].mean().rename(columns = {'S_return_excess' : 'Average Return'}) * 12
    vol = df.groupby('regime')[['S_return_excess']].std().rename(columns = {'S_return_excess' : 'Volatility'}) * np.sqrt(12)
    regime_df = excess_ret.join(vol)
    regime_df['Sharpe'] = regime_df['Average Return'] / regime_df['Volatility']
    display(regime_df.round(3))
    return regime_df

print("1M")
_ = summarize_regime(options_clean_1M_final)

print("1Y")
_ = summarize_regime(options_clean_1Y_final)

1M


,Average Return,Volatility,Sharpe
regime,,,
High Vol,-0.862,1.831,-0.471
Low Vol,-1.848,1.650,-1.120


1Y


,Average Return,Volatility,Sharpe
regime,,,
High Vol,0.017,0.285,0.060
Low Vol,-0.058,0.214,-0.273


#### Discussion: We observe that: (1) straddles benefits from high volatility regimes and suffer significantly in the low volatility regime. This is a very intuitive result in line with straddles as a strategy profits off of high volatility (2) Straddles tend to yield higher average excess return and high Sharpe raio for longer maturity. 

### 2. The variance risk premium and the equity risk premium

#### 2.a Download Variance Risk Premium

#### 2.b Download cum-dividend return from DataStream on S&P 500 and risk-free rate.

In [43]:
ret = db.raw_sql("""SELECT DISTINCT  valuedate as date, EXTRACT(YEAR FROM valuedate) as year, EXTRACT(MONTH FROM valuedate) as month, pi_ as pi, ri FROM tr_ds_equities.ds2indexdata WHERE dsindexcode=41620 AND ri IS NOT NULL ORDER BY valuedate""")

# Convert to monthly using end-of-month values
ret = ret.sort_values('date').groupby(['year', 'month']).last().sort_index()

ret = ret.reset_index()
ret = pd.merge(ret, rf.drop(columns = ['date']), on = ['year', 'month'], how = 'left')

ret['date'] = pd.to_datetime(ret['date'])
ret = ret.drop(columns = ['pi'])
ret = ret[ret['rf'].notna()].copy()

ret['ret'] = (ret['ri'] / ret['ri'].shift() - 1)
ret['excess_ret_1'] = ret['ret'] - ret['rf']

ret['excess_ret_3'] = ret['excess_ret_1'].rolling(3).apply(lambda x: np.prod(1 + x) - 1)
ret['excess_ret_12'] = ret['excess_ret_1'].rolling(12).apply(lambda x: np.prod(1 + x) - 1)

ret_merged = pd.merge(ret, vrp[['year', 'month', 'vrp', 'iv']], on = ['year', 'month'], how = 'right')

#### 2.c Report predictive regressions of excess returns on the variance risk premium for monthly, quarterly, and annual returns for two samples (in case of lower frequencies, use overlapping data):

In [44]:
def run_reg(data, freq, limit_yr, method = 'OLS'):
    data = data.copy(deep =True)
    data = data[data['year'] <= limit_yr]

    data['excess_ret_{}'.format(freq)] = data['excess_ret_{}'.format(freq)] * (12/freq)
    data['excess_ret_{}'.format(freq)] = data['excess_ret_{}'.format(freq)].shift(-freq)
    
    if method == 'OLS':
        data = data[['excess_ret_{}'.format(freq), 'vrp' ]].dropna()
        model = sm.OLS(data['excess_ret_{}'.format(freq)], sm.add_constant(data['vrp'])).fit()
    elif method == 'GLS':
        data = data[['excess_ret_{}'.format(freq), 'vrp', 'iv']].dropna()
        data['iv_Scaled'] = data['iv'] / 100
        model = sm.WLS(data['excess_ret_{}'.format(freq)], sm.add_constant(data['vrp']), weights = 1 / data['iv_Scaled']).fit()
    return model

i. 1990.1-2007.12.

In [45]:
model_list_GFC = [run_reg(ret_merged, freq, 2007) for freq in [1,3, 12]]
print("From 1990 till 2007", end = "\n\n")
print(summary_col(model_list_GFC, model_names =['Monthly', 'Quarterly', 'Annual'], stars = True), end = "\n\n")

From 1990 till 2007


               Monthly  Quarterly   Annual 
-------------------------------------------
const          -0.0022  -0.0181   0.0530***
               (0.0503) (0.0278)  (0.0160) 
vrp            0.0041*  0.0050*** 0.0015** 
               (0.0021) (0.0011)  (0.0006) 
R-squared      0.0176   0.0814    0.0261   
R-squared Adj. 0.0130   0.0770    0.0213   
Standard errors in parentheses.
* p<.1, ** p<.05, ***p<.01



ii. 1990.1-2023.12.

In [48]:
model_list_All = [run_reg(ret_merged, freq, 2023) for freq in [1,3, 12]]
print("From 1990 till 2023", end = "\n\n")
print(summary_col(model_list_All, model_names =['Monthly', 'Quarterly', 'Annual'], stars = True), end = "\n\n")

From 1990 till 2023


                Monthly  Quarterly   Annual 
--------------------------------------------
const          0.0849*** 0.0739*** 0.0948***
               (0.0290)  (0.0167)  (0.0093) 
vrp            0.0004    0.0011**  -0.0000  
               (0.0009)  (0.0005)  (0.0003) 
R-squared      0.0005    0.0125    0.0000   
R-squared Adj. -0.0021   0.0099    -0.0027  
Standard errors in parentheses.
* p<.1, ** p<.05, ***p<.01



#### 2.d So far, we have estimated the predictive regressions using OLS. However, we can also estimate predictive regressions using GLS. As a simple version, use the squared VIX as the weight in weighted least squares. That is, the estimator solves

$$
(\beta_0, \beta_1) = \arg \min_{\beta_0,\beta_1} \sum_{t=1}^{T-1} \left( \frac{R^e_{t+1} - \beta_0 - \beta_1VRP_t}{VIX^2_t} \right)^2
$$

#### Report the same results as above for month, quarterly, and annual returns and discuss whether GLS estimation is preferred and whether it is economically sensible.

In [84]:
model_list_GFC = [run_reg(ret_merged, freq, 2007, 'GLS') for freq in [1,3, 12]]
print("From 1990 till 2007", end = "\n\n")
print(summary_col(model_list_GFC, model_names =['Monthly', 'Quarterly', 'Annual'], stars = True), end = "\n\n")

From 1990 till 2007


               Monthly  Quarterly   Annual 
-------------------------------------------
const          0.0366   0.0314    0.0782***
               (0.0438) (0.0255)  (0.0143) 
vrp            0.0024   0.0028*   0.0008   
               (0.0027) (0.0016)  (0.0009) 
R-squared      0.0037   0.0156    0.0047   
R-squared Adj. -0.0010  0.0110    -0.0002  
Standard errors in parentheses.
* p<.1, ** p<.05, ***p<.01



In [85]:
model_list_All = [run_reg(ret_merged, freq, 2023, 'GLS') for freq in [1,3, 12]]
print("From 1990 till 2023", end = "\n\n")
print(summary_col(model_list_All, model_names =['Monthly', 'Quarterly', 'Annual'], stars = True), end = "\n\n")

From 1990 till 2023


               Monthly  Quarterly   Annual 
-------------------------------------------
const          0.0663** 0.0576*** 0.0871***
               (0.0277) (0.0168)  (0.0091) 
vrp            0.0012   0.0016*   0.0004   
               (0.0016) (0.0010)  (0.0005) 
R-squared      0.0014   0.0072    0.0016   
R-squared Adj. -0.0013  0.0046    -0.0011  
Standard errors in parentheses.
* p<.1, ** p<.05, ***p<.01



When we apply Generalized Least Squares (GLS) estimation to our predictive regressions, employing the squared VIX as weights, we observe a notable attenuation in the strength of our results. The predictive coefficient, which was significant in the OLS regressions, often loses significance under the GLS approach. This trend manifests across monthly, quarterly, and annual returns, as can be seen in the provided results. The implications of these findings are twofold:

First, from an econometric standpoint, GLS estimation is typically favored for its efficiency gains when heteroskedasticity is present. In theory, by weighting observations with the inverse of their variance, GLS accounts for changes in volatility and should provide more reliable estimates. However, the empirical evidence suggests that the weighting mechanism of GLS, which emphasizes periods of lower volatility due to the inverse relationship with the VIX, may dilute the impact of the variance risk premium (VRP).

Second, economically, the weakening of the results under GLS hints at a more nuanced relationship between VRP and excess returns. Specifically, it implies that the VRP is a more potent predictor during times of high market volatility. This could be interpreted as the VRP capturing a 'fear premium' that investors require to bear risk during turbulent market periods. Such periods often coincide with greater uncertainty and risk, leading to a more pronounced VRP effect.

Considering the apparent inconsistency between OLS and GLS results, a potential reconciliation could involve a model that accounts for regime changes, acknowledging that the predictive power of VRP may intensify during volatile times. Therefore, while the GLS estimator is theoretically more efficient, its practical application may be limited if it underweights the very conditions where the predictor variable—VRP—is most relevant.